<a href="https://colab.research.google.com/github/binliu0630/Data_Manipulation/blob/master/Copy_of_2020_03_21_covid19_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:
#hide

# FETCH
import getpass
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
base_url = '' if (getpass.getuser() == 'Pratap Vardhan') else base_url
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + 'overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [5]:
paths['overview']

'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/overview.tpl'

In [6]:
paths['mapping']

'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/mapping_countries.csv'

In [8]:
mapping = get_mappings(paths['mapping'])
mapping['df']

,Country,Name,Continent,Group1,Color
0,China,China,Asia,NaN,NaN
1,Italy,Italy,Europe,NaN,NaN
2,Iran,Iran,Asia,NaN,NaN
3,Spain,Spain,Europe,NaN,NaN
4,"Korea, South",South Korea,Asia,NaN,NaN
...,...,...,...,...,...
151,Suriname,Suriname,South America,NaN,NaN
152,Tanzania,Tanzania,Africa,NaN,NaN
153,The Bahamas,The Bahamas,North America,NaN,NaN
154,Togo,Togo,Africa,NaN,NaN


In [9]:
mapping['replace.country']

{'Afghanistan': 'Afghanistan',
 'Albania': 'Albania',
 'Algeria': 'Algeria',
 'Andorra': 'Andorra',
 'Antigua and Barbuda': 'Antigua and Barbuda',
 'Argentina': 'Argentina',
 'Armenia': 'Armenia',
 'Aruba': 'Aruba',
 'Australia': 'Australia',
 'Austria': 'Austria',
 'Azerbaijan': 'Azerbaijan',
 'Bahrain': 'Bahrain',
 'Bangladesh': 'Bangladesh',
 'Belarus': 'Belarus',
 'Belgium': 'Belgium',
 'Benin': 'Benin',
 'Bhutan': 'Bhutan',
 'Bolivia': 'Bolivia',
 'Bosnia and Herzegovina': 'Bosnia and Herzegovina',
 'Brazil': 'Brazil',
 'Brunei': 'Brunei',
 'Bulgaria': 'Bulgaria',
 'Burkina Faso': 'Burkina Faso',
 'Cambodia': 'Cambodia',
 'Cameroon': 'Cameroon',
 'Canada': 'Canada',
 'Central African Republic': 'Central African Republic',
 'Chile': 'Chile',
 'China': 'China',
 'Colombia': 'Colombia',
 'Congo (Brazzaville)': 'Congo (Brazzaville)',
 'Congo (Kinshasa)': 'Congo (Kinshasa)',
 'Costa Rica': 'Costa Rica',
 "Cote d'Ivoire": "Cote d'Ivoire",
 'Croatia': 'Croatia',
 'Cruise Ship': 'Cruise S

In [11]:
mapping['map.continent']

{'Afghanistan': 'Asia',
 'Albania': 'Europe',
 'Algeria': 'Africa',
 'Andorra': 'Europe',
 'Antigua and Barbuda': 'North America',
 'Argentina': 'South America',
 'Armenia': 'Asia',
 'Aruba': 'South America',
 'Australia': 'Oceania',
 'Austria': 'Europe',
 'Azerbaijan': 'Asia',
 'Bahrain': 'Asia',
 'Bangladesh': 'Asia',
 'Belarus': 'Europe',
 'Belgium': 'Europe',
 'Benin': 'Africa',
 'Bhutan': 'Asia',
 'Bolivia': 'South America',
 'Bosnia and Herzegovina': 'Europe',
 'Brazil': 'South America',
 'Brunei': 'Asia',
 'Bulgaria': 'Europe',
 'Burkina Faso': 'Africa',
 'Cambodia': 'Asia',
 'Cameroon': 'Africa',
 'Canada': 'North America',
 'Central African Republic': 'Africa',
 'Chile': 'South America',
 'China': 'Asia',
 'Colombia': 'South America',
 'Congo (Brazzaville)': 'Africa',
 'Congo (Kinshasa)': 'Africa',
 'Costa Rica': 'North America',
 "Cote d'Ivoire": 'Africa',
 'Croatia': 'Europe',
 'Cruise Ship': nan,
 'Cuba': 'North America',
 'Cyprus': 'Asia',
 'Czechia': 'Europe',
 'Denmark':

In [14]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22,22,22,24,24,40,40,74,84,94
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55,59,64,70,76,89,104,123,146,174
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60,74,87,90,139,201,230,264,302,367
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39,39,53,75,88,113,133,164,188,224
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,3,3,3,4


In [17]:
LAST_DATE_I, dt_cols = get_dates(df)
LAST_DATE_I, dt_cols

(-1, Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
        '1/28/20', '1/29/20', '1/30/20', '1/31/20', '2/1/20', '2/2/20',
        '2/3/20', '2/4/20', '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20',
        '2/10/20', '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20',
        '2/16/20', '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20',
        '2/22/20', '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20',
        '2/28/20', '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
        '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20', '3/12/20',
        '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20', '3/18/20',
        '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20', '3/24/20',
        '3/25/20', '3/26/20'],
       dtype='object'))

In [20]:

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]
dt_today, dt_5ago

('3/26/20', '3/21/20')

In [0]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfc_recovered = dft_recovered.groupby(COL_REGION)[dft_recovered.columns[-1]].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()
dfp_recovered = dft_recovered.groupby(COL_REGION)[dft_recovered.columns[-6]].sum()

In [24]:
dfc_cases.head(2)

Country/Region
Afghanistan     94
Albania        174
Name: 3/26/20, dtype: int64

In [23]:
dfc_deaths.head(2)

Country/Region
Afghanistan    4
Albania        6
Name: 3/26/20, dtype: int64

In [25]:
dfc_recovered.head(2)

Country/Region
Afghanistan     2
Albania        17
Name: 3/26/20, dtype: int64

# ADD STATE AND COUNTY

In [13]:
# Add US STATE AND COUNTY
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-25-2020.csv'
df = pd.read_csv(url)
df.head()


,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-25 23:33:19,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-25 23:33:19,30.295065,-92.414197,2,0,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-25 23:33:19,37.767072,-75.632346,2,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-25 23:33:19,43.452658,-116.241552,24,0,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-25 23:33:19,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [127]:
data = df[(df['Country_Region']=='US')&(df['Province_State'].isin(['Arkansas','New York', 'Louisiana']))]
df_state = data.groupby(['Province_State']).agg({'Confirmed':'sum', 'Deaths':'sum', 'Recovered':'sum'})
df_state

,Confirmed,Deaths,Recovered
Province_State,,,
Arkansas,280,2,0
Louisiana,1795,65,0
New York,30841,285,0


In [0]:
dfc_cases = dfc_cases.reset_index()

In [0]:
dfc_state = df_state[['Confirmed']].reset_index()
dfc_state.columns = dfc_cases.columns

In [0]:
dfc_cases = pd.concat([dfc_cases[dfc_cases['Country/Region'].isin(['US', 'China'])], dfc_state], axis=0)

In [0]:
dfc_cases.set_index('Country/Region', inplace=True)

In [0]:
dfc_cases = dfc_cases.iloc[:,0]

In [133]:
dfc_cases

Country/Region
China        81661
US           65778
Arkansas       280
Louisiana     1795
New York     30841
Name: 3/25/20, dtype: int64

In [135]:
type(dfc_cases)

pandas.core.series.Series

In [0]:
dfc_deaths = dfc_deaths.reset_index()
dfc_state = df_state[['Deaths']].reset_index()
dfc_state.columns = dfc_deaths.columns
dfc_deaths = pd.concat([dfc_deaths[dfc_deaths['Country/Region'].isin(['US', 'China'])], dfc_state], axis=0)
dfc_deaths.set_index('Country/Region', inplace=True)

In [0]:
dfc_deaths = dfc_deaths.iloc[:,0]

In [0]:
dfc_recovered = dfc_recovered.reset_index()
dfc_state = df_state[['Recovered']].reset_index()
dfc_state.columns = dfc_recovered.columns
dfc_recovered = pd.concat([dfc_recovered[dfc_recovered['Country/Region'].isin(['US', 'China'])], dfc_state], axis=0)
dfc_recovered.set_index('Country/Region', inplace=True)

In [0]:
dfc_recovered = dfc_recovered.iloc[:, 0]

In [0]:
#hide
dfc_recovered = dft_cases.copy()
dfc_recovered.loc[:, dft_cases.columns[4:]] = 0
dfc_recovered

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
0,NaN,Afghanistan,33.000000,65.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,Libya,26.335100,17.228331,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
241,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
242,NaN,Guinea-Bissau,11.803700,-15.180400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
243,NaN,Mali,17.570692,-3.996166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:

dfp_cases = dfc_cases
dfp_deaths = dfc_deaths
dfp_recovered = dfc_recovered

In [0]:
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered)).sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())

In [29]:
df_table

,Country/Region,Cases,Deaths,Recovered
0,US,83836,1209,681
1,China,81782,3291,74181
2,Italy,80589,8215,10361
3,Spain,57786,4365,7015
4,Germany,43938,267,5673
...,...,...,...,...
170,Somalia,2,0,0
171,Libya,1,0,0
172,Papua New Guinea,1,0,0
173,Saint Vincent and the Grenadines,1,0,0


In [0]:
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)

In [31]:
df_table

,Country/Region,Cases,Deaths,Recovered
0,US,83836,1209,681
1,China,81782,3291,74181
2,Italy,80589,8215,10361
3,Spain,57786,4365,7015
4,Germany,43938,267,5673
...,...,...,...,...
170,Somalia,2,0,0
171,Libya,1,0,0
172,Papua New Guinea,1,0,0
173,Saint Vincent and the Grenadines,1,0,0


In [144]:
#hide
df_table = (pd.DataFrame(dict(
    Cases=dfc_cases, Deaths=dfc_deaths, Recovered=dfc_recovered,
    PCases=dfp_cases, PDeaths=dfp_deaths, PRecovered=dfp_recovered))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
# for c in 'Cases, Deaths, Recovered'.split(', '):
#     df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
# df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
# df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
# df_table.head(15)

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81661,3285,73280,81661,3285,73280,0,0,0,4.0,Asia
1,US,65778,942,348,65778,942,348,0,0,0,1.4,North America
2,New York,30841,285,0,30841,285,0,0,0,0,0.9,NaN
3,Louisiana,1795,65,0,1795,65,0,0,0,0,3.6,NaN
4,Arkansas,280,2,0,280,2,0,0,0,0,0.7,NaN


In [0]:
#hide

#delete problematic countries from table
df_table = df_table[df_table['Country/Region'] != 'Cape Verde']
df_table = df_table[df_table['Country/Region'] != 'Cruise Ship']
df_table = df_table[df_table['Country/Region'] != 'Kosovo']

In [33]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Recovered', 'Cases (+)', 'Deaths (+)', 'Recovered (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
#s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu}
summary

KeyError: ignored

In [147]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20
Country/Region,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,2,0,0,4,5,5,1,0,0,2,0,16,0,34,10
Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,8,2,11,10,5,4,9,4,4,5,6,6,13,15,19,23
Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,2,7,0,5,0,2,1,0,0,4,2,11,11,6,6,14,13,3,49,62,29,34,38
Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,37,0,14,22,13,25,20,31,24
Angola,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0


In [148]:
df_table.head()

,Country/Region,Cases,Deaths,Recovered,PCases,PDeaths,PRecovered,Cases (+),Deaths (+),Recovered (+),Fatality Rate,Continent
0,China,81661,3285,73280,81661,3285,73280,0,0,0,4.0,Asia
1,US,65778,942,348,65778,942,348,0,0,0,1.4,North America
2,New York,30841,285,0,30841,285,0,0,0,0,0.9,NaN
3,Louisiana,1795,65,0,1795,65,0,0,0,0,3.6,NaN
4,Arkansas,280,2,0,280,2,0,0,0,0,0.7,NaN


In [166]:
#hide_input
template = Template(get_template(paths['overview']))
html = template.render(
    D=summary, table=df_table.head(2),
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],

    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')

<template>:141: RuntimeWarning: invalid value encountered in long_scalars


AttributeError: ignored

In [154]:
Template(get_template(paths['overview']))

<Template memory:7f451bab4be0>

In [155]:
paths['overview']

'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/overview.tpl'

In [0]:
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [0]:

#hide
from pathlib import Path
if not Path('covid_overview.py').exists():
    ! wget https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/covid_overview.py

--2020-03-26 17:14:45--  https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/covid_overview.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3810 (3.7K) [text/plain]
Saving to: ‘covid_overview.py’

covid_overview.py   100%[===================>]   3.72K  --.-KB/s    in 0s      

2020-03-26 17:14:46 (38.5 MB/s) - ‘covid_overview.py’ saved [3810/3810]



In [0]:
import covid_overview as covid

COL_REGION = 'Province/State'   
# Confirmed, Recovered, Deaths
US_POI = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
    'Colorado', 'Connecticut', 'Delaware', 'Diamond Princess',
    'District of Columbia', 'Florida', 'Georgia', 'Grand Princess',
    'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
    'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
    'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
    'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island',
    'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
    'Vermont', 'Virgin Islands', 'Virginia', 'Washington',
    'West Virginia', 'Wisconsin', 'Wyoming']

filter_us = lambda d: d[d['Country/Region'].eq('US') & d['Province/State'].isin(US_POI)]

kpis_info = [
    {'title': 'New York', 'prefix': 'NY'},
    {'title': 'Washington', 'prefix': 'WA'},
    {'title': 'California', 'prefix': 'CA'}]

data = covid.gen_data(region=COL_REGION, filter_frame=filter_us, kpis_info=kpis_info)

HTTPError: ignored

Visualizations by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

[^1]: Source: ["COVID-19 Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-overview.ipynb), [orignal interactive](https://gramener.com/enumter/covid19/)